In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
train_size  = None
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_test = x_train.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
layers = tf.keras.layers

In [9]:
gaussian_initializer=tf.keras.initializers.GlorotUniform()

In [10]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(img_input)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
out = layers.Dense(10, kernel_initializer=gaussian_initializer)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [11]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-4
    elif epoch < 55:
        return 1e-2
    elif epoch < 105:
        return 1e-2
    else:
        return 1e-4

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback])

In [15]:
model.load_weights('./model_weights/vgg_19_train=None_ce_init=gaussian.h5')

In [ ]:
model.save('./model_weights/vgg_19_train=None_ce_init=gaussian.h5')

In [16]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128])

tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test)

tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test)

==========NTK============
4/4 [==============================] - 0s 92ms/step - loss: 2.5115 - accuracy: 0.6250
==========CE============
64/64 [==============================] - 0s 4ms/step - loss: 2.4049 - accuracy: 0.5752
==========MSE============
64/64 [==============================] - 0s 4ms/step - loss: 2.2557 - accuracy: 0.5825


[2.255735158920288, 0.58251953125]

In [17]:
model.evaluate(x_test, y_test)

64/64 [==============================] - 0s 4ms/step - loss: 1.7362 - accuracy: 0.6670


[1.7362200021743774, 0.6669921875]

In [16]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128])

4/4 [==============================] - 1s 266ms/step - loss: 2.2308 - accuracy: 0.6328


[2.2307534217834473, 0.6328125]